# ChatData

## Sqlite
We are using a file system called **sqlite**. It looks and acts like a real single user relational database (RDB). sqlite3 comes packaged with python. You do not need to install the library. You simply import it (as below).

- We are using Pandas to write and read to sqlite. Pandas will manage a lot of the complexity of dealing with a RDB. There are other ways of reading and writing to a RDB that are VERY common and often used in production systems.  

- The most common other way is to read or write to the RDB row by row. As you can see Pandas puts the entire dataframe into the RDB or extracts a new dataframe from the RDB. These are not row by row operations. These are set operations. Set operations ARE more efficient. However, it is common to use row by row operations to avoid needing large memory computers to hold the dataframes. One row only takes a few bytes. An entire dataframe could be many gigabytes and even petabytes of data. Obviously you will end up with problems with your compute resource if your files are this big. This will not happen to you in this activity and may never happen to you while you are a data analyst. Please just be aware of this. 

- This activity is primarily about querying a RDB, so, we are using a simple way that Pandas provides. If you want to learn the more complex way, just google 'reading and wrting to a relational database using python'. There are many resources to learn from.

- The data types that sqlite supports are quite limited. for example, it does not have a DATE type.  This is not a challenge for this project. However, more sophisticated database systems,such as Postgresql, have a large array of data types, such as "DATE", etc. that give those systems additional capability. 

- All of the SQL queries could also be performed on the Pandas DataFrames directly.  You may want to try this yourself for comparison (but make sure you do the SQL queries first, as this is an exercise in using SQL!).


## SQL Magic
Within the Jupyter notebook we will be using something called **SQL Magic**.  This provides a convenient way to write SQL queries directly into code cells in the notebook and to read the results back into a Pandas DataFrame.  This makes working with SQL much easier!

Note that you may need to update your sqlite version using `conda update -c anaconda sqlite` in order for the SQL Magic to work correctly.

## The Data Analysis Lifecycle
The sections in this notebook follow the stages of the Data Analysis Lifecycle introduced in an earlier activity.  The stages are:

- Acquire
- Transform
- Organise
- Analyse
- Communicate
- Maintain


# Preliminary Steps: Create a Database
First let's import Sqlite and the other libraries we will need.

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import sqlite3
import requests
from io import BytesIO
from zipfile import ZipFile

### Create the Database
Now we will create the Sqlite database.  Here is some code that does this for you.  We use the `%load_ext` magic command to load the SQL Magic extension and then use `%sql` to connect to the database.

The requirements document for this project is Template SQL queries.xlsx.

In [2]:
# If the db does not exist, sqlite will create it.
con = sqlite3.connect('chatdata.db')

# loads sql magic
%load_ext sql 

# connects sql magic command to the correct db
%sql sqlite:///chatdata.db 

#### Drop the `queries` table if it already exists
The queries table will be our record of the queries created to answer the questions from the requirements spreadsheet.  As we will be running this Jupyter notebook a few times, let's drop (i.e. remove) the queries table so that we start fresh each time.  Here is the code to do this.  We use the `%%sql` magic command to tell Jupyter that we are going to write SQL in the cell.


In [3]:
%%sql
DROP TABLE IF EXISTS 'queries'

 * sqlite:///chatdata.db
Done.


[]

# Task 1: Load the Data
Now we can start loading the data.  The tables will be created as we load the data.  

Note that some of these files are quite large, so make sure you have plenty of free memory!

## Lifecycle Stages: Transform and Organise
The data has already been processed into 3 clean data files ready for this project:

- queries.csv
- posts.csv
- comments.csv

We will load these files into our database.

## Lifecycle Stage: Acquire
The data can be found in the OpenClassrooms instructions for this activity.

### Load Comments Data into a `comments` table
Now we will load the data from the csv files into our sqlite database.

First we load the csv file into a Pandas dataframe:

In [4]:
zip_file_url = "https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/DAN_UK_App_P5/chatdata.zip"
response = requests.get(zip_file_url)

In [5]:
if response.status_code == 200:
    with ZipFile(BytesIO(response.content)) as z:
        comments = pd.read_csv(z.open('comments.csv'))
        print(comments.head())
else:
    print(f"Failed to fetch the zip file. Status code: {response.status_code}")

       Id  PostId  Score                                               Text  \
0  723182  385124      0  @BenBolker I don't understand. The fit cannot ...   
1  723183  385124      3  You can't add *less* than (`-min(y)`), but you...   
2  723186  385137      0  nice. If you felt like doing the work it would...   
3  723187  385137      0  i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...   
4  723188  385134      0  Don't you mean "so variance should be $\sigma^...   

          CreationDate  UserId  
0  2019-01-01 00:06:39   78575  
1  2019-01-01 00:09:22    2126  
2  2019-01-01 00:32:11    2126  
3  2019-01-01 00:40:36    2126  
4  2019-01-01 00:41:28  112141  


Now take the comments dataframe and push the data into the Sqlite database table called 'comments':

In [6]:
# load comments into sqlite
comments.to_sql('comments', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM comments'
comments = pd.read_sql(sql, con)
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


### Load Other CSVs
Now that you have seen the code for loading in the comments.csv now do the same to read and write the posts.csv and users.csv to sqlite. 

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter code in the following cells. Insert as many cells as you want to do this.
    </font></div>

In [7]:
if response.status_code == 200:
    with ZipFile(BytesIO(response.content)) as z:
        posts = pd.read_csv(z.open('posts.csv'))
        print(posts.head())
else:
    print(f"Failed to fetch the zip file. Status code: {response.status_code}")

       Id  PostTypeId  AcceptedAnswerId  ParentId         CreationDate  Score  \
0  423497           1            423511         0  2019-08-24 09:39:31      2   
1  423498           1                 0         0  2019-08-24 09:47:42      1   
2  423499           1                 0         0  2019-08-24 09:48:26      1   
3  423500           2                 0    215865  2019-08-24 09:57:01      0   
4  423502           2                 0    423286  2019-08-24 10:44:52      3   

   ViewCount                                               Body  OwnerUserId  \
0         68  <p>From wikipedia <a href="https://en.wikipedi...        64552   
1         24  <p>I am currently doing local sensitivity anal...        87231   
2         56  <p>I'm an honours student in psychology doing ...       257207   
3          0  <p>Maybe you can look this <a href="https://me...       106606   
4          0  <blockquote>\n  <p>Q1) Is my approach valid?</...       220643   

  OwnerDisplayName  ...  LastEdi

In [8]:
# load posts into sqlite
posts.to_sql('posts', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM posts'
posts = pd.read_sql(sql, con)
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,None,...,None,None,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,None
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,None,...,None,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,None,None
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,None,...,None,None,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,None,None
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,None,...,None,None,2019-08-24 09:57:01,None,None,0,0,0,None,None
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,None,...,None,2019-08-24 12:13:46,2019-08-24 12:13:46,None,None,0,1,0,None,None


In [9]:
if response.status_code == 200:
    with ZipFile(BytesIO(response.content)) as z:
        users = pd.read_csv(z.open('users.csv'))
        print(users.head())
else:
    print(f"Failed to fetch the zip file. Status code: {response.status_code}")

       Id  Reputation         CreationDate DisplayName       LastAccessDate  \
0  157607          31  2017-04-17 14:50:42  user157607  2019-07-23 16:44:08   
1  157656         101  2017-04-17 20:08:20  user102859  2019-06-26 13:42:13   
2  157704         133  2017-04-18 05:10:47      jupiar  2019-11-25 13:32:27   
3  157709         155  2017-04-18 06:39:18      farmer  2019-02-17 19:44:24   
4  157755         101  2017-04-18 12:56:17      Miki P  2019-08-12 17:02:21   

  WebsiteUrl         Location  \
0        NaN              NaN   
1        NaN              NaN   
2        NaN  Shanghai, China   
3        NaN              NaN   
4        NaN              NaN   

                                             AboutMe  Views  UpVotes  \
0                                                NaN      0        0   
1                                                NaN      3        0   
2  <p>Originally from the U.K, I have an Undergra...      1        1   
3                                     

In [10]:
# load comments into sqlite
users.to_sql('users', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM users'
users = pd.read_sql(sql, con)
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,None,None,None,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,None,None,None,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,None,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,None,None,None,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,None,None,None,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


### **TODO**: Drop Duplicates
Look for and drop any duplicates in all 3 of the tables (if they exist). Use Pandas to do this. If you find duplicates, you will need to rewrite the table.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter code in the following cells. 
    </font></div>

In [11]:
comments_duplicates = comments[comments.duplicated("Id")]
print(f"Duplicate comments rows: {comments_duplicates}")

Duplicate comments rows: Empty DataFrame
Columns: [Id, PostId, Score, Text, CreationDate, UserId]
Index: []


In [12]:
users_duplicates = users[users.duplicated("Id")]
print(f"Duplicate comments rows: {users_duplicates}")

Duplicate comments rows: Empty DataFrame
Columns: [Id, Reputation, CreationDate, DisplayName, LastAccessDate, WebsiteUrl, Location, AboutMe, Views, UpVotes, DownVotes, ProfileImageUrl, AccountId]
Index: []


In [13]:
posts_duplicates = posts[posts.duplicated("Id")]
print(f"Duplicate comments rows: {posts_duplicates}")

Duplicate comments rows: Empty DataFrame
Columns: [Id, PostTypeId, AcceptedAnswerId, ParentId, CreationDate, Score, ViewCount, Body, OwnerUserId, OwnerDisplayName, LastEditorUserId, LastEditorDisplayName, LastEditDate, LastActivityDate, Title, Tags, AnswerCount, CommentCount, FavoriteCount, ClosedDate, CommunityOwnedDate]
Index: []

[0 rows x 21 columns]


## Review the Data

Spend some time reviewing the data.  Understand what data we have, think about how that data can be used to assist in the initiative of understanding how ChatData is used in the real world.  Is the data organised in a way that would lend itself to being manged in a relational database?  How would the different tables be connected?  What are the primary and foreign keys?  Would this give you a 3NF model?

Also think about security and ethics.  Is there personal data in here?  Could individuals be identified through this data?  It it ethical to use the data in this way?  You will be asked to comment on these questions later!

Use the code below to help you.

In [14]:
users.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'AccountId'],
      dtype='object')

In [15]:
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,None,None,None,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,None,None,None,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,None,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,None,None,None,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,None,None,None,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


In [16]:
comments.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId'], dtype='object')

In [17]:
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


In [18]:
posts.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'],
      dtype='object')

In [19]:
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,None,...,None,None,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,None
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,None,...,None,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,None,None
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,None,...,None,None,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,None,None
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,None,...,None,None,2019-08-24 09:57:01,None,None,0,0,0,None,None
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,None,...,None,2019-08-24 12:13:46,2019-08-24 12:13:46,None,None,0,1,0,None,None


## Writing queries with SQL Magic

You will now need to write some queries to get answers to the questions in the requirements.  

For single-line queries, start the cell with `%sql` and simply enter your query:

In [20]:
# This is an example
%sql SELECT COUNT(*) FROM comments

 * sqlite:///chatdata.db
Done.


COUNT(*)
50000


For multi line sql statements use `%%sql` as follows. This tells Jupyter that *everything* in this cell should be interpreted as sql. So, NO comments other statements are allowed:

In [21]:
%%sql
SELECT Id, PostId, Score, Text
    FROM comments
        LIMIT 5

 * sqlite:///chatdata.db
Done.


Id,PostId,Score,Text
723182,385124,0,"@BenBolker I don't understand. The fit cannot be done for the negative $y$. So intuitively I'd think that in order to retain the relativity of the data, one'd ideally ""mirror"" or ""flip"" it to the positive axis? Would it be possible to adjust every point individually? Basically e.g. take $abs(y_i)$ instead of minimums."
723183,385124,3,"You can't add *less* than (`-min(y)`), but you could add *more*. I'm going to stop answering now sorry, because **judging what the 'best' approach is depends on much more context that we don't have** (the goal of the analysis, why you need to fit an exponential, your level of computational and statistical sophistication and that of your audience, etc. ...)"
723186,385137,0,"nice. If you felt like doing the work it would be nice to generate an image/contour plot of log-likelihood as a function of (mu, theta) and show the lines corresponding to the two `size` values ."
723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,size=size,log=TRUE)), xlim=c(2,5),ylim=c(0.2,0.5), varnames=c(""mu"",""size""), sys3d=""contour"")abline(h=3.2/7,col=""red"")abline(h=fit.what$theta,col=""blue"")` ... although doesn't look this is actually the answer - mu-hat is independent of theta-hat ... ?"
723188,385134,0,"Don't you mean ""so variance should be $\sigma^2/(n\mu^2)$"""


## Writing queries with Pandas
Another way to write queries is to use pandas:

In [22]:
sql = """
SELECT Id, PostId, Score, Text
    FROM comments
        LIMIT 5
        """
result = pd.read_sql(sql, con)
result

,Id,PostId,Score,Text
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you..."
2,723186,385137,0,nice. If you felt like doing the work it would...
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si..."
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^..."


# Creating Tables with Referential Integrity

When we loaded the csv files into Sqlite database tables, Sqlite created the tables for us behind the scenes. Let's inspect this a bit more.

We can see how Sqlite created the tables by querying the `sqlite_master` table, which Sqlite uses to keep track of what objects have been created in the database:

In [23]:
%%sql
select sql from sqlite_master

 * sqlite:///chatdata.db
Done.


sql
"CREATE TABLE ""comments"" (""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER)"
"CREATE TABLE ""posts"" (""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT)"
"CREATE TABLE ""users"" (""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER)"


The above results show the `CREATE TABLE` statements that could be used by Sqlite to recreate the tables with the exact same structure.

The problem with the CREATE TABLE statements above is that they don't enforce **referential integrity**.  In other words, they don't ensure that every UserId and PostId in the comments table refers to an actual UserId and PostId in the users and posts tables.  At the moment. we can insert any old number here, and even have multiple users with the same Id!  One of the advantages of working with relational databases is that they can enforce the correct uniqueness and relationships in the data, but at the moment we are not using that feature.  So let's fix that...

First, let's drop the original tables:


In [24]:
%%sql
DROP TABLE comments;
DROP TABLE users;
DROP TABLE posts;

 * sqlite:///chatdata.db
Done.
Done.
Done.


[]

Prove that this worked by selecting the names of the tables back.  We should have no tables:

In [25]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'
    ORDER BY name

 * sqlite:///chatdata.db
Done.


name


In Sqlite we need to enable the enforcement of foreign key constraints:

In [26]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///chatdata.db
Done.


[]

Now recreate the users table with a **primary key constraint** by copying the CREATE TABLE statement from above and adding the NOT NULL PRIMARY KEY clause to the Id:

In [27]:
%%sql
CREATE TABLE "users" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"Reputation" INTEGER,
"CreationDate" TEXT,
"DisplayName" TEXT,
"LastAccessDate" TEXT,
"WebsiteUrl" TEXT,
"Location" TEXT,
"AboutMe" TEXT,
"Views" INTEGER,
"UpVotes" INTEGER,
"DownVotes" INTEGER,
"ProfileImageUrl" TEXT,
"AccountId" INTEGER
);


 * sqlite:///chatdata.db
Done.


[]

Now do the same for the posts table:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [28]:
posts.dtypes

Id                        int64
PostTypeId                int64
AcceptedAnswerId          int64
ParentId                  int64
CreationDate             object
Score                     int64
ViewCount                 int64
Body                     object
OwnerUserId               int64
OwnerDisplayName         object
LastEditorUserId          int64
LastEditorDisplayName    object
LastEditDate             object
LastActivityDate         object
Title                    object
Tags                     object
AnswerCount               int64
CommentCount              int64
FavoriteCount             int64
ClosedDate               object
CommunityOwnedDate       object
dtype: object

In [29]:
%%sql
CREATE TABLE "posts" (
    "Id" INTEGER NOT NULL PRIMARY KEY,
    "PostTypeId" INTEGER,
    "AcceptedAnswerId" INTEGER,
    "ParentId" INTEGER,
    "CreationDate" TEXT,
    "Score" INTEGER,
    "ViewCount" INTEGER,
    "Body" TEXT,
    "OwnerUserId" INTEGER,
    "OwnerDisplayName" TEXT,
    "LastEditorUserId" INTEGER,
    "LastEditorDisplayName" TEXT,
    "LastEditDate" TEXT,
    "LastActivityDate" TEXT,
    "Title" TEXT,
    "Tags" TEXT,
    "AnswerCount" INTEGER,
    "CommentCount" INTEGER,
    "FavoriteCount" INTEGER,
    "ClosedDate" TEXT,
    "CommunityOwnedDate" TEXT
); 

 * sqlite:///chatdata.db
Done.


[]

Now for the comments table.  We need to add the primary key constraint on the id here as we did for users and posts, but we also need to add FOREIGN KEY constraints on the UserId and PostId. Read the documentation here and find our how to do that: https://www.sqlite.org/foreignkeys.html.  Then create the comments table with the correct constraints:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [30]:
comments.dtypes

Id               int64
PostId           int64
Score            int64
Text            object
CreationDate    object
UserId           int64
dtype: object

In [31]:
%%sql
CREATE TABLE "comments" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostId" INTEGER,
"Score" INTEGER,
"Text" TEXT,
"CreationDate" TEXT,
"UserId" INTEGER
); 

 * sqlite:///chatdata.db
Done.


[]

Now we can re-insert the data into these constrained tables.  First users:

In [32]:
# Insert data into the new users table
users.to_sql('users', con, if_exists='append', index=False)

18412

Now posts:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [33]:
# Insert data into the new posts table
posts.to_sql('posts', con, if_exists='append', index=False)

42234

Finally comments, which references the users and posts tables:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [34]:
# Insert data into the new comments table
comments.to_sql('comments', con, if_exists='append', index=False)

50000

Now check that we have the 3 new table definitions in Sqlite:

In [35]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'
    ORDER BY name

 * sqlite:///chatdata.db
Done.


name
comments
posts
users


We now have all the data in tables in Sqlite and the tables will enforce the referential integrity.

# Example Query and Pattern for Tasks 2 and 3

As you work through the next tasks, you will need to:

1. Prepare the Sqlite query to answer the question
2. Test it
3. Insert it into the `queries` table, so we have a record of it for others.

This is the process that we want you to follow for this project while completing it. 

Let's see an example of this by answering the following question:

**Which 5 users have viewed the most times and what is the sum of those views per user?**

## Prepare the Sqlite query

First, let's write the query:

In [36]:
sql = """
SELECT Id, SUM(Views) AS TotalViews
    FROM Users
        GROUP BY Id
            ORDER BY TotalViews DESC
                LIMIT 5
                """
result = pd.read_sql(sql, con) # con is the connection to the database
result

,Id,TotalViews
0,919,85180
1,4253,35119
2,805,34637
3,7290,32639
4,3277,29255


## Test the query
You can optionally prove the query worked by performing the same query in Pandas:

In [37]:
results = users.groupby(['Id']).sum().sort_values('Views', ascending = False)[:5]
results['Views']

Id
919     85180
4253    35119
805     34637
7290    32639
3277    29255
Name: Views, dtype: int64

## Insert the query into the `queries` table
Now we need to put this query into the `queries` table in sqlite. Remember we want these queries to be accessible to everybody that should have access to them. We do not want people writing and rewriting the same queires over and over again. The easiest thing to do is create a dictionary with the values and insert these into the queries table.  Note that the values are provided as lists as we are inserting a list of values (i.e. a number of rows) into the table.  In this case the number of rows is 1, so we have lists of 1 item.

So here, we have a column called 'task' with a list of values, a column called 'action' with a list of values, etc.

In [38]:
query_dict = {
              'task': ['Single Table Queries'],
              'action': ['Which 5 users have viewed the most times and what is the sum of those views per user?'],
              'query': [sql]
             }
query_dict

{'task': ['Single Table Queries'],
 'action': ['Which 5 users have viewed the most times and what is the sum of those views per user?'],
 'query': ['\nSELECT Id, SUM(Views) AS TotalViews\n    FROM Users\n        GROUP BY Id\n            ORDER BY TotalViews DESC\n                LIMIT 5\n                ']}

Now that you have the data structure (query_dict) containing the data, create a pandas dataframe that holds those values:

In [39]:
queries = pd.DataFrame(query_dict)
queries

,task,action,query
0,Single Table Queries,Which 5 users have viewed the most times and w...,"\nSELECT Id, SUM(Views) AS TotalViews\n FRO..."


Now load that pandas dataframe (queries) into the sqlite table called queries. In this case, you use append NOT replace. You will be adding to this tables as you go thru this project.

In [40]:
# load query into sqlite
queries.to_sql('queries', con, if_exists='append', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM queries'
queries = pd.read_sql(sql, con)
queries.head()

,task,action,query
0,Single Table Queries,Which 5 users have viewed the most times and w...,"\nSELECT Id, SUM(Views) AS TotalViews\n FRO..."


It is likely that as you iterate thru this notebook you will create some duplicate entries in the query table. Not a big deal. Just tell sql or pandas (much easier in pandas) to drop the duplicates! If you drop the duplicates in Pandas you will have to write the entire dataframe back to sqlite. Otherwise it is changed in memory in Pandas but NOT in sqlite on the disk drive (or SDD). Just do this drop at the end of this notebook so that you are not constantly dealing with this. 

So, to summarise, as you go through the following tasks you need to:

- answer the question in sql
- prove it in pandas (if you want to)
- put the query into the queries table

# Task 1 (continued): Insert the CREATE TABLE Statements into the `queries` Table

Now that we understand how to populate the `queries` table, let's insert the CREATE TABLE statements into it.  First let's define a function to help us insert into the queries table:


In [41]:
# Define a function that will insert into the queries table
def store_query(task, action, new_query):
    query_dict_chat = {
                  'task': [task],
                  'action': [action],
                  'query': [new_query]
                 }

    # put query into the query_dict
    chat_queries = pd.DataFrame(query_dict_chat)

    # load query into sqlite
    chat_queries.to_sql('queries', con, if_exists='append', index=False)    

Now we can specify the queries and call the above function to store them.  The first one is done for you.

In [42]:
sql_query = """
    CREATE TABLE "comments" (
    "Id" INTEGER,
    "PostId" INTEGER,
    "Score" INTEGER,
    "Text" TEXT,
    "CreationDate" TEXT,
    "UserId" INTEGER
    )
    """
store_query("Task 1", "Get the CREATE TABLE statement for Comments", sql_query)

Let's prove it works by selecting back from the queries table:

In [43]:
# Prove it works
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Get the CREATE TABLE statement for Comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"


## Insert the other CREATE TABLE statements into the `queries` table.
Follow the above pattern to complete these code cells:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cells
    </font></div>

In [44]:
# Insert the CREATE TABLE for posts into the queries table
sql_query = """
    CREATE TABLE "posts" (
    "Id" INTEGER,
    "PostTypeId" INTEGER,
    "AcceptedAnswerId" INTEGER,
    "ParentId" INTEGER,
    "CreationDate" TEXT,
    "Score" INTEGER,
    "ViewCount" INTEGER,
    "Body" TEXT,
    "OwnerUserId" INTEGER,
    "OwnerDisplayName" TEXT,
    "LastEditorUserId" INTEGER,
    "LastEditorDisplayName" TEXT,
    "LastEditDate" TEXT,
    "LastActivityDate" TEXT,
    "Title" TEXT,
    "Tags" TEXT,
    "AnswerCount" INTEGER,
    "CommentCount" INTEGER,
    "FavoriteCount" INTEGER,
    "ClosedDate" TEXT,
    "CommunityOwnedDate" TEXT
    )
    """
store_query("Task 1", "Get the CREATE TABLE statement for Posts", sql_query)

In [45]:
# Insert the CREATE TABLE for users into the queries table
sql_query = """
    CREATE TABLE "users" (
    "Id" INTEGER,
    "Reputation" INTEGER,
    "CreationDate" TEXT,
    "DisplayName" TEXT,
    "LastAccessDate" TEXT,
    "WebsiteUrl" TEXT,
    "Location" TEXT,
    "AboutMe" TEXT,
    "Views" INTEGER,
    "UpVotes" INTEGER,
    "DownVotes" INTEGER,
    "ProfileImageUrl" TEXT,
    "AccountId" INTEGER
    )
    """
store_query("Task 1", "Get the CREATE TABLE statement for Users", sql_query)

In [46]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Get the CREATE TABLE statement for Comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 1,Get the CREATE TABLE statement for Posts,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 1,Get the CREATE TABLE statement for Users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"


## Count the Number of Rows in Each Table
Run some queries to count the number of rows in each of the tables.  Don't forget to insert the query into the queries table.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cells
    </font></div>

In [47]:
# Count the number of rows in the comments table
sql_query = """
    SELECT COUNT(*) FROM comments
"""
store_query("Task 1", "Count number of rows in table Comments", sql_query)

In [48]:
# Count the number of rows in the users table
sql_query = """
    SELECT COUNT(*) FROM users
"""
store_query("Task 1", "Count number of rows in table Users", sql_query)

In [49]:
# Count the number of rows in the posts table
sql_query = """
    SELECT COUNT(*) FROM posts
"""
store_query("Task 1", "Count number of rows in table Posts", sql_query)

In [50]:
# check queries
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Get the CREATE TABLE statement for Comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 1,Get the CREATE TABLE statement for Posts,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 1,Get the CREATE TABLE statement for Users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 1,Count number of rows in table Comments,SELECT COUNT(*) FROM comments
Task 1,Count number of rows in table Users,SELECT COUNT(*) FROM users
Task 1,Count number of rows in table Posts,SELECT COUNT(*) FROM posts


## Do some Random Checks on the Data
Let's write some queries that select 5 random rows from each table.  The queries are provided here:

    select * from Comments order by random() limit 5;
    select * from Users order by random() limit 5;
    select * from Posts order by random() limit 5;

Enter the queries into the code cells below and insert the queries into the `queries` table in the same was as you did for the CREATE TABLE statements.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cells
    </font></div>

# Working with Sqlite and SQL Magic
In this section let's spend a little time understanding a bit more about how we can work with Sqlite within Jupyter.

Let's look at 2 ways to query the sqlite database: using SQL Magic or using Pandas.  Either way is fine for this project.

In [51]:
# Run the query to select 5 random rows from the posts table
sql_query = """
    SELECT * FROM posts order by random() limit 5
"""
store_query("Task 1", "Five random posts", sql_query)

In [52]:
# Run the query to select 5 random rows from the posts comments
sql_query = """
    SELECT * FROM comments order by random() limit 5
"""
store_query("Task 1", "Five random comments", sql_query)

In [53]:
# Run the query to select 5 random rows from the users table
sql_query = """
    SELECT * FROM users order by random() limit 5
"""
store_query("Task 1", "Five random users", sql_query)

In [54]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Get the CREATE TABLE statement for Comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 1,Get the CREATE TABLE statement for Posts,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 1,Get the CREATE TABLE statement for Users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 1,Count number of rows in table Comments,SELECT COUNT(*) FROM comments
Task 1,Count number of rows in table Users,SELECT COUNT(*) FROM users
Task 1,Count number of rows in table Posts,SELECT COUNT(*) FROM posts
Task 1,Five random posts,SELECT * FROM posts order by random() limit 5
Task 1,Five random comments,SELECT * FROM comments order by random() limit 5
Task 1,Five random users,SELECT * FROM users order by random() limit 5


# Task 2: Create Single Table Queries


## Lifecycle Stage: Analyze
We can now start the analysis with our single-table queries.  First we need to create a new computed column to help with one of the queries.  The code below creates a column called LEN_BODY which is the length of the BODY text:


In [55]:
%%sql
ALTER TABLE POSTS ADD COLUMN LEN_BODY INT

 * sqlite:///chatdata.db
Done.


[]

In [56]:
%%sql
UPDATE POSTS SET LEN_BODY = LENGTH(BODY)

 * sqlite:///chatdata.db
42234 rows affected.


[]

### Single Table Queries
From the Template SQL Queries there are a series of queries requested. You are responsible for coding the result. Use the pattern above to accomplish the coding objective.  Copy and paste the question (column B Action) into a markdown cell and answer the query. Use the pattern that you have been using above. 

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter your code below.  Just keep on inserting cells as you need them.
    </font></div>

How many posts have 0 comments?

In [57]:
sql_query = """
SELECT COUNT(*)
FROM posts
WHERE CommentCount = 0
        """
store_query("Task 2", "How many posts have 0 comments?", sql_query)
result = pd.read_sql(sql_query, con)
result

,COUNT(*)
0,21713


In [58]:
query = """
SELECT COUNT(*)
FROM posts
WHERE CommentCount = 1
        """
store_query("Task 2", "How many posts have 1 comments?", query)
result = pd.read_sql(query, con)
result

,COUNT(*)
0,6460


In [59]:
sql_query = """
SELECT COUNT(*)
FROM posts
WHERE CommentCount >= 2
        """
store_query("Task 2", "How many posts have 2 comments or more?", sql_query)
result = pd.read_sql(sql_query, con)
result

,COUNT(*)
0,14061


In [60]:
sql_query = """
SELECT *
FROM posts
ORDER BY ViewCount DESC
    LIMIT 5
        """
store_query("Task 2", "Find the 5 posts with the highest viewcount", sql_query)
result = pd.read_sql(sql_query, con)
result

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,LEN_BODY
0,388566,1,388582,0,2019-01-22 15:16:47,56,19542,"<ul>\n<li>Statement One (S1): ""One in 80 death...",228214,None,...,2019-01-22 21:40:39,2019-01-24 17:09:47,"Is it wrong to rephrase ""1 in 80 deaths is cau...",<interpretation><risk>,9,15,15,None,None,2270
1,394118,1,394128,0,2019-02-24 14:07:11,64,16317,<p>A human child at age 2 needs around 5 insta...,107213,None,...,2019-02-25 22:40:22,2019-03-03 17:37:05,Why do neural networks need so many training e...,<neural-networks><neuroscience>,12,24,38,None,None,512
2,431370,1,431397,0,2019-10-14 11:29:21,77,11723,<p>It seems very counter intuitive to many peo...,262594,None,...,2019-11-28 01:44:34,2019-11-28 01:44:34,Is there a name for the phenomenon of false po...,<probability><terminology><intuition>,8,9,18,None,None,811
3,398646,1,398653,0,2019-03-21 01:19:52,61,9850,<p>The title of the Comment in Nature <a href=...,163067,None,...,2019-03-22 22:14:04,2019-03-30 19:35:27,"What does ""Scientists rise up against statisti...",<statistical-significance><p-value><bias>,10,7,34,None,None,2148
4,434128,1,434579,0,2019-11-01 13:07:36,73,6718,<p>I am designing a one year program in data a...,14188,None,...,None,2019-11-26 00:59:15,Famous statistical wins and horror stories for...,<mathematical-statistics><data-visualization><...,13,7,70,None,2019-11-01 15:12:41,1172


In [61]:
sql_query = """
SELECT 
    *
FROM 
    posts
ORDER BY 
    Score DESC
    LIMIT 5
        """
store_query("Task 2", "Find the top 5 posts with the highest scores", sql_query)
result = pd.read_sql(sql_query, con)
result

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,LEN_BODY
0,431397,2,0,431370,2019-10-14 14:29:36,101,0,<p>Yes there is. Generally it is termed <stron...,142976,None,...,None,2019-10-14 14:29:36,None,None,0,0,0,None,None,269
1,394128,2,0,394118,2019-02-24 15:44:44,100,0,<p>I caution against expecting strong resembla...,22311,None,...,2019-03-03 17:37:05,2019-03-03 17:37:05,None,None,0,15,0,None,None,5829
2,426878,2,0,426873,2019-09-11 23:23:31,93,0,<p><strong>tl;dr</strong> Even though this is ...,119015,None,...,2019-09-13 14:02:28,2019-09-13 14:02:28,None,None,0,6,0,None,None,4627
3,388578,2,0,388566,2019-01-22 15:48:47,80,0,"<p>To me ""1 in 80 deaths..."" is by far the cle...",227039,None,...,None,2019-01-22 15:48:47,None,None,0,11,0,None,None,572
4,431370,1,431397,0,2019-10-14 11:29:21,77,11723,<p>It seems very counter intuitive to many peo...,262594,None,...,2019-11-28 01:44:34,2019-11-28 01:44:34,Is there a name for the phenomenon of false po...,<probability><terminology><intuition>,8,9,18,None,None,811


In [62]:
sql_query = """
SELECT 
    Score, COUNT(*) AS Frequency
FROM 
    posts
GROUP BY 
    Score  
ORDER BY 
    Frequency DESC, Score
    LIMIT 5
        """
store_query("Task 2", "What are the 5 most frequent scores on posts?", sql_query)
result = pd.read_sql(sql_query, con)
result

,Score,Frequency
0,0,19888
1,1,11867
2,2,5094
3,3,2228
4,4,1059


In [63]:
sql_query = """
SELECT 
    COUNT(*) AS NumPostsWithDataTag
FROM 
    posts
WHERE 
    Tags 
LIKE 
    '%data%' 
        """
store_query("Task 2", "How many posts have the keyword 'data' in their tags?", sql_query)
result = pd.read_sql(sql_query, con)
result

,NumPostsWithDataTag
0,2242


In [64]:
sql_query = """
SELECT 
    commentcount, COUNT(*) AS Frequency
FROM 
    posts
GROUP BY 
    commentcount  
ORDER BY 
    Frequency DESC, commentcount
    LIMIT 5
        """
store_query("Task 2", "What are the 5 most frequent commentcount for posts?", sql_query)
result = pd.read_sql(sql_query, con)
result

,CommentCount,Frequency
0,0,21713
1,1,6460
2,2,4966
3,3,3063
4,4,2026


In [65]:
sql_query = """
SELECT 
    COUNT(*)
FROM 
    posts
WHERE 
    AcceptedAnswerId >0
        """
store_query("Task 2", "How many posts have an accepted answer?", sql_query)
result = pd.read_sql(sql_query, con)
result

,COUNT(*)
0,5341


In [66]:
sql_query = """
SELECT 
    AVG(Reputation) AS AverageReputation
FROM 
    users
"""
store_query("Task 2", "What is the average reputation of table users?", sql_query)
result = pd.read_sql(sql_query, con)
result

,AverageReputation
0,312.350912


In [67]:
sql_query = """
SELECT 
    MIN(Reputation) AS MinReputation,
    MAX(Reputation) AS MaxReputation
FROM 
    users
"""
store_query("Task 2", "What are the min and max reputation of users?", sql_query)
result = pd.read_sql(sql_query, con)
result

,MinReputation,MaxReputation
0,1,228662


In [68]:
sql_query = """
SELECT 
    Id,
    LENGTH(Body) AS BodyLength
FROM 
    posts
ORDER BY 
    ViewCount DESC
LIMIT 5
"""
store_query("Task 2", "What is the length of the body of 5 most viewed posts?", sql_query)
result = pd.read_sql(sql_query, con)
result

,Id,BodyLength
0,388566,2270
1,394118,512
2,431370,811
3,398646,2148
4,434128,1172


In [69]:
sql_query = """
SELECT 
    COUNT(DISTINCT Location) AS NumDistinctLocations
FROM 
    users
"""
store_query("Task 2", "How many different locations are there in the users table?", sql_query)
result = pd.read_sql(sql_query, con)
result

,NumDistinctLocations
0,1900


In [70]:
sql_query = """
SELECT 
    Location,
    COUNT(*) AS UserCount
FROM 
    users
WHERE Location IS NOT NULL AND Location != ''
GROUP BY 
    Location
ORDER BY 
    UserCount DESC
LIMIT 5
"""
store_query("Task 2", "What are the top 5 locations of users?", sql_query)
result = pd.read_sql(sql_query, con)
result

,Location,UserCount
0,Germany,117
1,India,100
2,United States,69
3,"Paris, France",66
4,"London, United Kingdom",63


In [71]:
sql_query = """
WITH DayOfWeekViewCounts AS (
    SELECT 
        CASE
            WHEN STRFTIME('%w', CreationDate) = '0' THEN 'Sunday'
            WHEN STRFTIME('%w', CreationDate) = '1' THEN 'Monday'
            WHEN STRFTIME('%w', CreationDate) = '2' THEN 'Tuesday'
            WHEN STRFTIME('%w', CreationDate) = '3' THEN 'Wednesday'
            WHEN STRFTIME('%w', CreationDate) = '4' THEN 'Thursday'
            WHEN STRFTIME('%w', CreationDate) = '5' THEN 'Friday'
            WHEN STRFTIME('%w', CreationDate) = '6' THEN 'Saturday'
        END AS DayOfWeek,
        SUM(ViewCount) AS TotalViewCount
    FROM 
        posts
    GROUP BY 
        DayOfWeek
)

SELECT 
    DayOfWeek,
    TotalViewCount,
    (TotalViewCount * 100.0 / SUM(TotalViewCount) OVER ()) AS Percentage
FROM 
    DayOfWeekViewCounts
ORDER BY 
    Percentage DESC
"""
store_query("Task 2", "Rank the days of the week from highest to lowest in terms of the volume of ViewCount as a percentage.", sql_query)
result = pd.read_sql(sql_query, con)
result

,DayOfWeek,TotalViewCount,Percentage
0,Thursday,331507,16.819476
1,Wednesday,330506,16.768689
2,Tuesday,320382,16.255034
3,Monday,311546,15.806727
4,Friday,267324,13.563061
5,Sunday,234459,11.895609
6,Saturday,175247,8.891404


In [72]:
# check queries
%sql SELECT * FROM queries


 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Get the CREATE TABLE statement for Comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 1,Get the CREATE TABLE statement for Posts,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 1,Get the CREATE TABLE statement for Users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 1,Count number of rows in table Comments,SELECT COUNT(*) FROM comments
Task 1,Count number of rows in table Users,SELECT COUNT(*) FROM users
Task 1,Count number of rows in table Posts,SELECT COUNT(*) FROM posts
Task 1,Five random posts,SELECT * FROM posts order by random() limit 5
Task 1,Five random comments,SELECT * FROM comments order by random() limit 5
Task 1,Five random users,SELECT * FROM users order by random() limit 5


# Task 3: Cross Table Queries

## Lifecycle Stage: Analyze
Let's continue the analysis with our multi-table queries.  

### Cross Table Queries

From the Template SQL Queries there are a series of queries requested. You are responsible for coding the result. Use the pattern above to accomplish the coding objective.  Copy and paste the question (column B Action) into a markdown cell and answer the query. Use the pattern that you have been using above. 

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter your code below.  Just keep on inserting cells as you need them.
    </font></div>



In [73]:
# TODO How many posts have been created by a user that has a filled out the "AboutMe" section?
sql_query = """
SELECT COUNT(*) AS NumPosts
FROM posts p
JOIN users u ON p.OwnerUserId = u.Id
WHERE u.AboutMe IS NOT NULL AND u.AboutMe != '';

"""
store_query("Task 3", "How many posts have been created by a user that has a filled out the 'AboutMe' section?", sql_query)
result = pd.read_sql(sql_query, con)
result

,NumPosts
0,17189


In [74]:
# TODO Considering only the users with an "AboutMe," how many posts are there per user?
sql_query = """
SELECT 
    COUNT(*) / (
        SELECT CAST(COUNT(*) AS FLOAT)
        FROM users
        WHERE AboutMe != ''
    ) AS AvgNumPosts
FROM 
    posts p
JOIN 
    users u ON u.Id = p.OwnerUserId
WHERE 
    u.AboutMe IS NOT NULL AND TRIM(u.AboutMe) != '';
"""
store_query("Task 3", "Considering only the users with an 'AboutMe,' how many posts are there per user?", sql_query)
result = pd.read_sql(sql_query, con)
result

,AvgNumPosts
0,4.274807


In [75]:
# TODO Not taking into account the CommentCount field in the table posts, what are the Top 10 posts in terms of number of comments?
sql_query = """
SELECT
    p.Id AS PostId,
    p.Score AS PostScore,
    COUNT(*) AS CCount
FROM
    posts p
LEFT JOIN
    comments c ON p.Id = c.PostId
GROUP BY
    p.Id
ORDER BY
    CCount DESC
LIMIT 10;
"""
store_query("Task 3", "Not taking into account the CommentCount field in the table posts, what are the Top 10 posts in terms of number of comments?", sql_query)
result = pd.read_sql(sql_query, con)
result

,PostId,PostScore,CCount
0,386853,1,66
1,386556,4,34
2,395232,-1,31
3,418910,35,31
4,402987,1,27
5,386075,10,26
6,394118,64,24
7,398828,6,23
8,402950,1,23
9,396111,1,22


In [76]:
# TODO What are the Top 10 posts which have the highest cumulative (post score + comment score) score? 
sql_query = """
SELECT
    p.Id AS PostId,
    p.Score AS PostScore,
    SUM(c.Score) AS CommentScore,
    SUM(p.Score) + SUM(c.Score) AS CumulativeScore,
    COUNT(c.Id) AS CommentCount
FROM
    posts p
LEFT JOIN
    comments c ON p.Id = c.PostId
GROUP BY
    p.Id, p.Score
ORDER BY
    CumulativeScore DESC
LIMIT 10;
"""
store_query("Task 3", "What are the Top 10 posts which have the highest cumulative (post score + comment score) score?", sql_query)
result = pd.read_sql(sql_query, con)
result

,PostId,PostScore,CommentScore,CumulativeScore,CommentCount
0,394118,64,242,1778,24
1,394128,100,69,1569,15
2,418910,35,9,1094,31
3,398653,65,46,1021,15
4,388578,80,61,941,11
5,388566,56,45,885,15
6,396818,44,16,808,18
7,418814,51,9,621,12
8,394258,39,24,570,14
9,394439,35,6,566,16


In [77]:
# TODO Who are the top 10 users who comment the most? 
sql_query = """
SELECT
    u.Id AS UserId,
    COUNT(*) AS Frequency
FROM
    comments c
JOIN
    users u ON c.UserId = u.Id
GROUP BY
    u.Id
ORDER BY
    Frequency DESC
LIMIT 10;
"""
store_query("Task 3", "Who are the top 10 users who comment the most?", sql_query)
result = pd.read_sql(sql_query, con)
result

,UserId,Frequency
0,919,3301
1,805,1153
2,143489,1024
3,11887,805
4,85665,691
5,164061,540
6,22047,536
7,158565,504
8,7962,492
9,35989,470


In [78]:
# TODO Who are the top 10 users who post the most? 
sql_query = """
SELECT
    u.Id AS OwnerUserId,
    COUNT(*) AS Frequency
FROM
    posts p
JOIN
    users u ON p.OwnerUserId = u.Id
GROUP BY
    u.Id
ORDER BY
    Frequency DESC
LIMIT 10;
"""
store_query("Task 3", "Who are the top 10 users who post the most?", sql_query)
result = pd.read_sql(sql_query, con)
result

,OwnerUserId,Frequency
0,204068,637
1,85665,545
2,173082,435
3,11887,435
4,686,386
5,1352,285
6,3382,274
7,7224,233
8,35989,230
9,805,230


# Task 4: Check the Queries Table

Now let's tidy up and check the queries table.

First let's check it's contents:

In [79]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Get the CREATE TABLE statement for Comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 1,Get the CREATE TABLE statement for Posts,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 1,Get the CREATE TABLE statement for Users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 1,Count number of rows in table Comments,SELECT COUNT(*) FROM comments
Task 1,Count number of rows in table Users,SELECT COUNT(*) FROM users
Task 1,Count number of rows in table Posts,SELECT COUNT(*) FROM posts
Task 1,Five random posts,SELECT * FROM posts order by random() limit 5
Task 1,Five random comments,SELECT * FROM comments order by random() limit 5
Task 1,Five random users,SELECT * FROM users order by random() limit 5


## Drop Duplicates

You likely have some duplicates. Lets drop them. 

In [80]:
# Read the queries table into pandas
sql = 'SELECT * FROM queries'
queries = pd.read_sql(sql, con)

# Drop duplicates
queries.drop_duplicates(inplace = True) # drop duplicates

In [81]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Get the CREATE TABLE statement for Comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 1,Get the CREATE TABLE statement for Posts,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 1,Get the CREATE TABLE statement for Users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 1,Count number of rows in table Comments,SELECT COUNT(*) FROM comments
Task 1,Count number of rows in table Users,SELECT COUNT(*) FROM users
Task 1,Count number of rows in table Posts,SELECT COUNT(*) FROM posts
Task 1,Five random posts,SELECT * FROM posts order by random() limit 5
Task 1,Five random comments,SELECT * FROM comments order by random() limit 5
Task 1,Five random users,SELECT * FROM users order by random() limit 5


In [82]:
%sql SELECT count(*) FROM Queries

 * sqlite:///chatdata.db
Done.


count(*)
31


## Case Issues

Remember, SQL is case insensitive. Pandas IS case sensitive. Lets deal with this now by making all of the text uppercase. 

In [83]:
for col in queries.columns:
    queries[col] = queries[col].str.upper()
    
queries

,task,action,query
0,SINGLE TABLE QUERIES,WHICH 5 USERS HAVE VIEWED THE MOST TIMES AND W...,"\nSELECT ID, SUM(VIEWS) AS TOTALVIEWS\n FRO..."
1,TASK 1,GET THE CREATE TABLE STATEMENT FOR COMMENTS,"\n CREATE TABLE ""COMMENTS"" (\n ""ID"" INTE..."
2,TASK 1,GET THE CREATE TABLE STATEMENT FOR POSTS,"\n CREATE TABLE ""POSTS"" (\n ""ID"" INTEGER..."
3,TASK 1,GET THE CREATE TABLE STATEMENT FOR USERS,"\n CREATE TABLE ""USERS"" (\n ""ID"" INTEGER..."
4,TASK 1,COUNT NUMBER OF ROWS IN TABLE COMMENTS,\n SELECT COUNT(*) FROM COMMENTS\n
5,TASK 1,COUNT NUMBER OF ROWS IN TABLE USERS,\n SELECT COUNT(*) FROM USERS\n
6,TASK 1,COUNT NUMBER OF ROWS IN TABLE POSTS,\n SELECT COUNT(*) FROM POSTS\n
7,TASK 1,FIVE RANDOM POSTS,\n SELECT * FROM POSTS ORDER BY RANDOM() LI...
8,TASK 1,FIVE RANDOM COMMENTS,\n SELECT * FROM COMMENTS ORDER BY RANDOM()...
9,TASK 1,FIVE RANDOM USERS,\n SELECT * FROM USERS ORDER BY RANDOM() LI...


In [84]:
# Write the now deduped uppercase dataframe back to sqlite and replace the table
queries.to_sql('queries', con, if_exists='replace', index=False)

31

## Use Case

Now that we have this queries table, lets give you some ideas about how you would use it.

Suppose you wanted to find all of the queries where you did a GROUP BY:

In [85]:
%%sql
SELECT query
    FROM queries
        WHERE query LIKE '%GROUP BY%'

 * sqlite:///chatdata.db
Done.


query
"SELECT ID, SUM(VIEWS) AS TOTALVIEWS FROM USERS GROUP BY ID ORDER BY TOTALVIEWS DESC LIMIT 5"
"SELECT SCORE, COUNT(*) AS FREQUENCYFROM POSTSGROUP BY SCORE ORDER BY FREQUENCY DESC, SCORE LIMIT 5"
"SELECT COMMENTCOUNT, COUNT(*) AS FREQUENCYFROM POSTSGROUP BY COMMENTCOUNT ORDER BY FREQUENCY DESC, COMMENTCOUNT LIMIT 5"
"SELECT LOCATION, COUNT(*) AS USERCOUNTFROM USERSWHERE LOCATION IS NOT NULL AND LOCATION != ''GROUP BY LOCATIONORDER BY USERCOUNT DESCLIMIT 5"
"WITH DAYOFWEEKVIEWCOUNTS AS ( SELECT CASE WHEN STRFTIME('%W', CREATIONDATE) = '0' THEN 'SUNDAY' WHEN STRFTIME('%W', CREATIONDATE) = '1' THEN 'MONDAY' WHEN STRFTIME('%W', CREATIONDATE) = '2' THEN 'TUESDAY' WHEN STRFTIME('%W', CREATIONDATE) = '3' THEN 'WEDNESDAY' WHEN STRFTIME('%W', CREATIONDATE) = '4' THEN 'THURSDAY' WHEN STRFTIME('%W', CREATIONDATE) = '5' THEN 'FRIDAY' WHEN STRFTIME('%W', CREATIONDATE) = '6' THEN 'SATURDAY' END AS DAYOFWEEK, SUM(VIEWCOUNT) AS TOTALVIEWCOUNT FROM POSTS GROUP BY DAYOFWEEK)SELECT DAYOFWEEK, TOTALVIEWCOUNT, (TOTALVIEWCOUNT * 100.0 / SUM(TOTALVIEWCOUNT) OVER ()) AS PERCENTAGEFROM DAYOFWEEKVIEWCOUNTSORDER BY PERCENTAGE DESC"
"SELECT P.ID AS POSTID, P.SCORE AS POSTSCORE, COUNT(*) AS CCOUNTFROM POSTS PLEFT JOIN COMMENTS C ON P.ID = C.POSTIDGROUP BY P.IDORDER BY CCOUNT DESCLIMIT 10;"
"SELECT P.ID AS POSTID, P.SCORE AS POSTSCORE, SUM(C.SCORE) AS COMMENTSCORE, SUM(P.SCORE) + SUM(C.SCORE) AS CUMULATIVESCORE, COUNT(C.ID) AS COMMENTCOUNTFROM POSTS PLEFT JOIN COMMENTS C ON P.ID = C.POSTIDGROUP BY P.ID, P.SCOREORDER BY CUMULATIVESCORE DESCLIMIT 10;"
"SELECT U.ID AS USERID, COUNT(*) AS FREQUENCYFROM COMMENTS CJOIN USERS U ON C.USERID = U.IDGROUP BY U.IDORDER BY FREQUENCY DESCLIMIT 10;"
"SELECT U.ID AS OWNERUSERID, COUNT(*) AS FREQUENCYFROM POSTS PJOIN USERS U ON P.OWNERUSERID = U.IDGROUP BY U.IDORDER BY FREQUENCY DESCLIMIT 10;"


## Now Your Turn
Find the queries that have 'DISTINCT' in them. You can do it with the %sql command or with Pandas and sql.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [86]:
%%sql
SELECT query
    FROM queries
        WHERE query LIKE '%DISTINCT%'

 * sqlite:///chatdata.db
Done.


query
SELECT COUNT(DISTINCT LOCATION) AS NUMDISTINCTLOCATIONSFROM USERS


# Close SQLite

It is good practise to close all relational databases as soon as you are finished updating them.

In [87]:
# con.close()

# All Done!

Great job. You now have a good idea for how to use sql and pandas with sql. You can create your own databases from csv files and you can do extensive querying using sql. These are valuable skills that will take you a long ways in todays technological world.